In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge

In [2]:
X_train = pd.read_csv('csv/X_train.csv')
X_test = pd.read_csv('csv/X_test.csv')
y_train = pd.read_csv('csv/y_train.csv')
y_test = pd.read_csv('csv/y_test.csv') 

In [3]:
model = Ridge(alpha=0.05)

In [4]:
model.fit(X_train,y_train)

Ridge(alpha=0.05)

In [5]:
model.score(X_test,y_test)

0.9999999999883448

In [6]:
prediction = model.predict(X_test)
prediction

array([[ 297000.44682317],
       [1577999.49503801],
       [ 562098.93739427],
       ...,
       [ 619998.93361598],
       [ 764999.88696632],
       [ 373000.99314221]])